In [ ]:
# 10. Вспомогательная функция для загрузки стоп-слов
def load_custom_stopwords(filepath='../custom_stopwords.json'):
    """
    Загружает кастомный список стоп-слов из JSON файла
    
    Returns:
        set: Множество стоп-слов
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return set(data['custom_stopwords'])

# Пример использования:
# stopwords = load_custom_stopwords()
# filtered_tokens = [token for token in tokens if token not in stopwords]

print("✓ Функция load_custom_stopwords() готова к использованию")

In [ ]:
# 9. Сохраняем кастомный список стоп-слов
import json

# Сохраняем в JSON для удобства использования
stopwords_data = {
    'custom_stopwords': sorted(list(custom_stopwords)),
    'russian_filtered': sorted(list(russian_stopwords_filtered)),
    'english': sorted(list(english_stopwords)),
    'kazakh': sorted(list(kazakh_stopwords)),
    'sentiment_critical': sorted(list(sentiment_critical_words)),
    'metadata': {
        'total_count': len(custom_stopwords),
        'russian_count': len(russian_stopwords_filtered),
        'english_count': len(english_stopwords),
        'kazakh_count': len(kazakh_stopwords),
        'description': 'Кастомный список стоп-слов с сохранением sentiment-важных слов'
    }
}

# Сохраняем
with open('../custom_stopwords.json', 'w', encoding='utf-8') as f:
    json.dump(stopwords_data, f, ensure_ascii=False, indent=2)

print("✓ Кастомный список стоп-слов сохранён в файл: ../custom_stopwords.json")
print(f"\nИтоговая статистика:")
print(f"  - Всего стоп-слов: {len(custom_stopwords)}")
print(f"  - Русских (фильтр.): {len(russian_stopwords_filtered)}")
print(f"  - Английских: {len(english_stopwords)}")
print(f"  - Казахских: {len(kazakh_stopwords)}")
print(f"  - Сохранено sentiment-critical: {len(sentiment_critical_words)}")

In [ ]:
# 8. Анализ топ-100 слов: что оставляем, что удаляем
print("АНАЛИЗ ТОП-100 ЧАСТОТНЫХ СЛОВ")
print("="*80)

# Разделяем слова из топ-100 на категории
to_remove = []  # Стоп-слова
to_keep_sentiment = []  # Важные для сентимента
to_keep_domain = []  # Важные для предметной области (рестораны)

for word, count in top_100:
    if word in custom_stopwords:
        to_remove.append((word, count))
    elif word in sentiment_critical_words or any(marker in word for marker in ['не', 'очень', 'но']):
        to_keep_sentiment.append((word, count))
    else:
        to_keep_domain.append((word, count))

print(f"\n1. УДАЛЯЕМ (стоп-слова): {len(to_remove)} слов")
print("   " + ", ".join([f"{w}({c})" for w, c in to_remove[:20]]))
if len(to_remove) > 20:
    print(f"   ... и ещё {len(to_remove)-20} слов")

print(f"\n2. СОХРАНЯЕМ (sentiment-важные): {len(to_keep_sentiment)} слов")
for w, c in to_keep_sentiment:
    print(f"   {w} ({c})")

print(f"\n3. СОХРАНЯЕМ (предметная область): {len(to_keep_domain)} слов")
print("   " + ", ".join([f"{w}({c})" for w, c in to_keep_domain[:30]]))
if len(to_keep_domain) > 30:
    print(f"   ... и ещё {len(to_keep_domain)-30} слов")

In [ ]:
# 7. Объединяем все стоп-слова в единый набор
custom_stopwords = russian_stopwords_filtered.union(english_stopwords).union(kazakh_stopwords)

print(f"Итоговый размер кастомного списка стоп-слов: {len(custom_stopwords)}")
print(f"  - Русские (фильтрованные): {len(russian_stopwords_filtered)}")
print(f"  - Английские: {len(english_stopwords)}")
print(f"  - Казахские: {len(kazakh_stopwords)}")

# Проверяем, что sentiment-critical слова не попали в стоп-слова
intersection = custom_stopwords.intersection(sentiment_critical_words)
if intersection:
    print(f"\n⚠️ ВНИМАНИЕ! Найдены sentiment-critical слова в стоп-словах: {intersection}")
else:
    print("\n✓ Проверка пройдена: sentiment-critical слова сохранены")

In [ ]:
# 6. Добавляем казахские стоп-слова на основе частотного анализа
# Казахские предлоги, союзы, местоимения и частицы

kazakh_stopwords_manual = {
    # Основные служебные слова казахского языка
    'және', 'бір', 'де', 'да', 'ол', 'бұл', 'сол', 'осы', 'міне', 'ме', 'ма', 
    'ба', 'бе', 'па', 'пе', 'не', 'кім', 'қай', 'қайда', 'қалай', 'қашан',
    'неге', 'не', 'немесе', 'не', 'әрі', 'тағы', 'та', 'дағы', 'деп', 'дейді',
    'үшін', 'себебі', 'сондықтан', 'өйткені', 'мен', 'сен', 'біз', 'сіз', 'олар',
    'менің', 'сенің', 'оның', 'біздің', 'сіздің', 'олардың', 'ма', 'ме', 'ба', 'бе',
    'па', 'пе', 'ша', 'ше', 'шы', 'ші', 'шық', 'шік', 'шақ', 'шек',
}

# Добавляем найденные из частотного анализа
kazakh_stopwords = kazakh_stopwords_manual.union(set(kazakh_stopwords_candidates))

print(f"Казахских стоп-слов (ручной список): {len(kazakh_stopwords_manual)}")
print(f"Казахских стоп-слов (из частотного анализа): {len(kazakh_stopwords_candidates)}")
print(f"Всего казахских стоп-слов: {len(kazakh_stopwords)}")

print("\nСписок казахских стоп-слов:")
print(sorted(kazakh_stopwords))

In [ ]:
# 5. Создание кастомного списка стоп-слов
import nltk
from nltk.corpus import stopwords

# Загружаем стандартные стоп-слова
try:
    russian_stopwords = set(stopwords.words('russian'))
except:
    nltk.download('stopwords')
    russian_stopwords = set(stopwords.words('russian'))

try:
    english_stopwords = set(stopwords.words('english'))
except:
    english_stopwords = set(stopwords.words('english'))

print(f"Стандартных русских стоп-слов: {len(russian_stopwords)}")
print(f"Стандартных английских стоп-слов: {len(english_stopwords)}")

# Слова, которые НЕЛЬЗЯ удалять (важны для сентимента)
sentiment_critical_words = {
    # Отрицания
    'не', 'нет', 'ни', 'без', 'нельзя', 'никогда', 'никто', 'ничто', 'никакой', 
    'нету', 'недо', 'небез',
    
    # Усилители
    'очень', 'совсем', 'слишком', 'весь', 'всё', 'все', 'самый', 'столь', 
    'чрезвычайно', 'крайне', 'особо', 'чересчур',
    
    # Противопоставительные союзы
    'но', 'однако', 'хотя', 'зато', 'впрочем',
    
    # Модальные слова
    'можно', 'нужно', 'должен', 'стоит', 'надо', 'следует'
}

# Исключаем sentiment-critical слова из русских стоп-слов
russian_stopwords_filtered = russian_stopwords - sentiment_critical_words

print(f"\nРусских стоп-слов после фильтрации: {len(russian_stopwords_filtered)}")
print(f"Исключено sentiment-critical слов: {len(sentiment_critical_words)}")

## Создание кастомного списка стоп-слов

Принципы формирования:
1. **Включаем стандартные русские стоп-слова**, НО исключаем важные для сентимент-анализа
2. **Включаем стандартные английские стоп-слова**
3. **Включаем казахские стоп-слова** на основе частотного анализа
4. **НЕ удаляем:**
   - Отрицания: не, нет, ни, без, нельзя
   - Усилители: очень, совсем, слишком, весь, всё, самый, столь
   - Противопоставительные союзы: но, однако, хотя, зато
   - Модальные слова: можно, нужно, должен, стоит

In [ ]:
# 4. Анализ общей частоты всех токенов (топ-100)
# Используем уже созданный ранее word_freq или пересоздаём
all_tokens = []
for tokens_list in df['tokens'].dropna():
    if isinstance(tokens_list, list):
        all_tokens.extend(tokens_list)

word_freq = Counter(all_tokens)
top_100 = word_freq.most_common(100)

print("ТОП-100 наиболее частотных слов во всех текстах:\n")
print(f"{'№':<5} {'Слово':<20} {'Частота':<10}")
print("="*35)
for i, (word, count) in enumerate(top_100, 1):
    print(f"{i:<5} {word:<20} {count:<10}")

In [ ]:
# 3. Определение казахских стоп-слов
# Будем считать слово казахским, если оно содержит казахские буквы
kazakh_letters = set('әіңғүұқөһӘІҢҒҮҰҚӨҺ')

def is_kazakh_word(word):
    """Проверяет, содержит ли слово казахские буквы"""
    return any(letter in kazakh_letters for letter in word)

# Фильтруем только казахские слова из топ-50
kazakh_stopwords_candidates = []
print("Кандидаты в казахские стоп-слова (из топ-50):\n")
print(f"{'№':<5} {'Слово':<20} {'Частота':<10}")
print("="*35)

counter = 1
for word, count in top_50_kk:
    if is_kazakh_word(word):
        kazakh_stopwords_candidates.append(word)
        print(f"{counter:<5} {word:<20} {count:<10}")
        counter += 1

print(f"\nНайдено казахских слов-кандидатов: {len(kazakh_stopwords_candidates)}")

In [ ]:
# 2. Частотный анализ казахских токенов (топ-50)
from collections import Counter

kk_word_freq = Counter(kk_tokens)
top_50_kk = kk_word_freq.most_common(50)

print("ТОП-50 наиболее частотных слов из казахских/смешанных текстов:\n")
print(f"{'№':<5} {'Слово':<20} {'Частота':<10}")
print("="*35)
for i, (word, count) in enumerate(top_50_kk, 1):
    print(f"{i:<5} {word:<20} {count:<10}")

In [ ]:
# 1. Извлечение токенов из казахских и смешанных текстов
print("Распределение текстов по типам:")
print(df['text_type'].value_counts())
print(f"\nВсего текстов: {len(df)}")

# Фильтруем тексты с казахским языком
kk_texts = df[df['text_type'].isin(['kazakh', 'mixed_kk_ru'])]
print(f"\nТекстов с казахским языком: {len(kk_texts)}")
print(f"  - kazakh: {len(df[df['text_type'] == 'kazakh'])}")
print(f"  - mixed_kk_ru: {len(df[df['text_type'] == 'mixed_kk_ru'])}")

# Извлекаем токены из казахских текстов
kk_tokens = []
for tokens_list in kk_texts['tokens'].dropna():
    if isinstance(tokens_list, list):
        kk_tokens.extend(tokens_list)

print(f"\nВсего токенов из казахских/смешанных текстов: {len(kk_tokens)}")
print(f"Уникальных токенов: {len(set(kk_tokens))}")

## Анализ казахских токенов для создания стоп-слов

В этом разделе мы:
1. Извлечём токены из текстов, классифицированных как 'mixed_kk_ru' или 'kazakh'
2. Проанализируем частоту казахских токенов
3. Покажем топ-30-50 наиболее частотных казахских слов
4. Проанализируем общую частоту токенов (топ-100)
5. Создадим кастомный список стоп-слов с сохранением слов, важных для сентимент-анализа

In [1]:
import pandas as pd

df = pd.read_csv('../Lova_cof.csv')
df

,place_firm_id,place_name,place_category,place_category_search,place_rating,place_address,author,author_reviews_count,rating,text,text_length,date,is_verified,sentiment
0,70000001057770550,"Lova Coffee, кофейня",Кофейни,кофейня,4.6,Не указан,Арайлым Куандык,2,5,От всей души благодарим официантку Гулжайну за...,150,2025-10-05,True,positive
1,70000001057770550,"Lova Coffee, кофейня",Кофейни,кофейня,4.6,Не указан,Merey ♥️,16,1,Обслуживание оставляет желать лучшего. Персона...,100,2025-10-05,True,negative
2,70000001057770550,"Lova Coffee, кофейня",Кофейни,кофейня,4.6,Не указан,Adina Nursapina,1,5,"очень милый персонал,осталась очень довольной😍😍😍",48,2025-10-03,True,positive
3,70000001057770550,"Lova Coffee, кофейня",Кофейни,кофейня,4.6,Не указан,Zhigerkizi demi,41,5,Реально вкусно и суперские милкшейки,36,2025-10-03,True,positive
4,70000001057770550,"Lova Coffee, кофейня",Кофейни,кофейня,4.6,Не указан,Михаил Класс,21,5,"Очень красивое и вкусное место, идеально подхо...",225,2025-10-03,True,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2307,70000001057770550,"Lova Coffee, кофейня",Кофейни,кофейня,4.6,Не указан,Айтолкын Жанкелді,15,1,"Вчера были у вас, интерьер все круто. Но обслу...",371,2021-12-28,True,negative
2308,70000001057770550,"Lova Coffee, кофейня",Кофейни,кофейня,4.6,Не указан,Артур Артур,14,1,"Пришли компанией в полупустой зал, ожидали око...",236,2021-12-26,True,negative
2309,70000001057770550,"Lova Coffee, кофейня",Кофейни,кофейня,4.6,Не указан,Карина Сапашева,4,2,"Были здесь вчера с подругами, впечатления оста...",386,2021-12-26,True,negative
2310,70000001057770550,"Lova Coffee, кофейня",Кофейни,кофейня,4.6,Не указан,Sary Qyz,64,5,5 поставлю за вкусную кухню👍🏻обслуживание было...,169,2021-12-24,True,positive


In [3]:
df.drop(columns=['place_firm_id', 'place_name', 'place_category',
       'place_category_search', 'place_rating', 'place_address', 'is_verified'], axis=1, inplace=True)

In [4]:
df

,author,author_reviews_count,rating,text,text_length,date,sentiment
0,Арайлым Куандык,2,5,От всей души благодарим официантку Гулжайну за...,150,2025-10-05,positive
1,Merey ♥️,16,1,Обслуживание оставляет желать лучшего. Персона...,100,2025-10-05,negative
2,Adina Nursapina,1,5,"очень милый персонал,осталась очень довольной😍😍😍",48,2025-10-03,positive
3,Zhigerkizi demi,41,5,Реально вкусно и суперские милкшейки,36,2025-10-03,positive
4,Михаил Класс,21,5,"Очень красивое и вкусное место, идеально подхо...",225,2025-10-03,positive
...,...,...,...,...,...,...,...
2307,Айтолкын Жанкелді,15,1,"Вчера были у вас, интерьер все круто. Но обслу...",371,2021-12-28,negative
2308,Артур Артур,14,1,"Пришли компанией в полупустой зал, ожидали око...",236,2021-12-26,negative
2309,Карина Сапашева,4,2,"Были здесь вчера с подругами, впечатления оста...",386,2021-12-26,negative
2310,Sary Qyz,64,5,5 поставлю за вкусную кухню👍🏻обслуживание было...,169,2021-12-24,positive


In [5]:
df.shape

(2312, 7)

In [6]:
df.columns.tolist()

['author',
 'author_reviews_count',
 'rating',
 'text',
 'text_length',
 'date',
 'sentiment']

In [10]:
df.text_length.describe()

count    2312.000000
mean      173.803201
std       211.300026
min        30.000000
25%        66.000000
50%       111.500000
75%       196.000000
max      2000.000000
Name: text_length, dtype: float64

In [31]:
df[df['text_length'] == 1999]

,author,author_reviews_count,rating,text,text_length,date,sentiment
262,Aidana K,3,1,Обслуживание ужасное! Сколько ходила в это зав...,1999,2025-06-23,negative


In [35]:
df['text'][262]

'Обслуживание ужасное! Сколько ходила в это заведение, любила очень! Всех гостей звала сюда, как раз сестры приехали и пришли посидеть, но увы испортили нам вечер!такого обслуживания никогда не встречала! Сидели в террасе только заказали, принесли нам еду и тут начался дождь, хотели пересесть туда где есть навес, подождали пока один стол там освободится. Место освободилось, вежливо попросили официантов убрать со стола, что хотим туда пересесть так как дождя там не было, а тут мы сидим под дождем. полный игнор со стороны официантов! 2 раза подошли к ним, просили нормально, в 00.45 сестренке сказали нет смысла уже пересаживаться, так как «уже они закроются». И я пошла во второй раз у официанта-парня прошу еще раз, в ответ мне он тоже говорит что уже закроются. спрашиваю во сколько отвечает в 1.45. Так у нас есть еще час я говорю! Мы даже чай начали только пить, мы бы могли сидеть еще до закрытия. Он говорит ну я вас просто предупреждаю. Зачем мне его предупреждения? Мы хотим просто посид

In [37]:
df.text.head(5)

0    От всей души благодарим официантку Гулжайну за...
1    Обслуживание оставляет желать лучшего. Персона...
2     очень милый персонал,осталась очень довольной😍😍😍
3                 Реально вкусно и суперские милкшейки
4    Очень красивое и вкусное место, идеально подхо...
Name: text, dtype: object

In [36]:
df['text'].isnull().sum()

np.int64(0)

In [38]:
import re

emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # смайлики
      u"\U0001F300-\U0001F5FF"  # символы
      u"\U0001F680-\U0001F6FF"  # транспорт
      u"\U0001F1E0-\U0001F1FF"  # флаги
      u"\U00002702-\U000027B0"  # дингбаты
      u"\U000024C2-\U0001F251"  # enclosed
      u"\U0001F900-\U0001F9FF"  # доп символы
      u"\U00002600-\U000026FF"  # разные символы (включая ❤️)
      "]+", flags=re.UNICODE)

In [40]:
has_emoji = df['text'].apply(lambda x: bool(emoji_pattern.search(x)))

print(has_emoji.sum(), has_emoji.sum()/len(df)*100)

1192 51.55709342560554


In [41]:
has_emoji.sum()/df.text_length.sum()*100

np.float64(0.2966406442477348)

In [42]:
df.text_length.sum()

np.int64(401833)

In [43]:
has_newline = df['text'].str.contains('\n', na=False)
has_newline.sum()

np.int64(435)

In [46]:
has_newline

0       False
1       False
2       False
3       False
4       False
        ...  
2307    False
2308     True
2309    False
2310    False
2311     True
Name: text, Length: 2312, dtype: bool

In [44]:
duplicates = df['text'].duplicated()
duplicates.sum()

np.int64(2)

In [49]:
example = df[has_newline]['text'].iloc[0]
example

'От всей нашей компании говорим спасибо официанту - Зарине за доброжелательное обслуживание. Она нас ещё и сфоткала🥰\n\nОбожаю Lova Coffee. Уже четвертый раз с друзьями здесь что-то отмечаем. Еда вообще вкусная. Когда пеперонни заказываешь, прям чувствуется сочный сыр. Скажите спасибо поварам. Это нереально всё вкусно. Однозначно, наше любимое заведение для атмосферных дружеских посиделок❤️'

In [50]:
example.replace('\n', ' ')

'От всей нашей компании говорим спасибо официанту - Зарине за доброжелательное обслуживание. Она нас ещё и сфоткала🥰  Обожаю Lova Coffee. Уже четвертый раз с друзьями здесь что-то отмечаем. Еда вообще вкусная. Когда пеперонни заказываешь, прям чувствуется сочный сыр. Скажите спасибо поварам. Это нереально всё вкусно. Однозначно, наше любимое заведение для атмосферных дружеских посиделок❤️'

In [52]:
import string 

def has_special_chars(text):
    normal = set(string.ascii_letters + string.digits + string.punctuation + ' ' + 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ' 
+ 'әіңғүұқөһӘІҢҒҮҰҚӨҺ')

    special = set(text) - normal - set(emoji_pattern.findall(text))

    return len(special) > 0, special

sample = df['text']
special_found = sample.apply(has_special_chars)

In [53]:
special_found.apply(lambda x: x[0]).sum()

np.int64(1106)

In [54]:
special_found

0                          (True, {‍, ❤, ️})
1                                (False, {})
2                                (True, {😍})
3                                (False, {})
4                                (False, {})
                        ...                 
2307                             (False, {})
2308    (True, {», ️, 🏻, «, ♂, ✌, ‍, 🧔, \n})
2309                             (False, {})
2310                          (True, {🏻, 👍})
2311                            (True, {\n})
Name: text, Length: 2312, dtype: object

In [55]:
def clean_text(text):
    text = text.replace('\n', ' ')

    text = re.sub(r'\s+', ' ', text)

    text = text.strip()

    return text

test_text = df[has_newline]['text'].iloc[0]

print(test_text[:150])

print(clean_text(test_text)[:150])

От всей нашей компании говорим спасибо официанту - Зарине за доброжелательное обслуживание. Она нас ещё и сфоткала🥰

Обожаю Lova Coffee. Уже четвертый
От всей нашей компании говорим спасибо официанту - Зарине за доброжелательное обслуживание. Она нас ещё и сфоткала🥰 Обожаю Lova Coffee. Уже четвертый 


In [57]:
df['text_1'] = df['text'].apply(clean_text)
print(df['text'].str.contains('\n').sum())
print('\n')
print(df['text_1'].str.contains('\n').sum())

435


0


In [58]:
all_emojis = []

for text in df['text']:
    found = emoji_pattern.findall(text)
    all_emojis.extend(found)

unique_emojis = set(all_emojis)
print(len(unique_emojis))
print(sorted(unique_emojis))

449
['☀️', '☀️💕👍🏻👍🏻👍🏻', '☕', '☕✨', '☕️', '☹️', '☺️', '☺️☺️', '☺️☺️❤️', '☺️🥰', '♀️', '♀️💯', '♀️😡', '♀️🤦', '♀️🤦🏻', '♂️', '♂️🤦', '♂️🧚', '♥️', '⚡️', '✅💯', '✊', '✊🏻', '✊🏽', '✊🏾', '✌🏻', '✨', '✨️', '✨️✨️', '✨👍🏻', '✨😍', '❗', '❗️', '❣️', '❤', '❤❤❤❤', '❤️', '❤️❤️', '❤️❤️❤️', '❤️❤️❤️❤️', '❤️❤️❤️❤️❤️', '❤️❤️❤️❤️👍🏼👍🏼👍🏼👍🏼', '❤️👍🏻', '❤️🔥🔥🔥🔥🔥', '❤️😍', '❤️😍🌺', '❤️😍😉', '❤️😻', '❤️🙌🏼', '❤️🙏🏻', '❤️🤌🏻', '❤️🤍', '❤️🤍❤️', '❤️🤗', '❤️🤭', '❤️🥰', '❤️🥰😍', '️', '🌄', '🌟', '🌸', '🌸💕', '🌹', '🌹🌹🌹', '🌹🌹🌹🌹', '🌺🌺🌺🌺', '🍕', '🍣', '🍤❤️', '🍮', '🍳', '🍴', '🍾', '🎊', '🎵', '🏻', '🏻☺️', '🏻❤️', '🏻👍', '🏻👍🏻', '🏻💛', '🏻🤍', '🏻🤤', '🏼', '🏽', '🏽☺️', '👋', '👌', '👌🏻', '👌🏻👌🏻', '👌🏼', '👌👌👌', '👌👍👍👍', '👍', '👍☺️', '👍✨', '👍❤️', '👍🏻', '👍🏻🍀', '👍🏻👍🏻', '👍🏻👍🏻❤️', '👍🏻👍🏻👍🏻', '👍🏻👍🏻👍🏻👍🏻', '👍🏻👍🏻👍🏻💐💐💐💐💐', '👍🏻👍🏻😻', '👍🏻🔥', '👍🏻😍', '👍🏻😘', '👍🏻🙂', '👍🏻🤌🏻', '👍🏻🤍', '👍🏻🧿☀️', '👍🏼', '👍🏼👍🏼', '👍🏼👍🏼👍🏼', '👍🏼👍🏼👍🏼👍🏼👍🏼', '👍🏼🔥', '👍🏽👍🏻', '👍🏽👍🏽👍🏽', '👍🏽🥰🥰', '👍🏾', '👍🏾👍🏾', '👍🏿👍🏿', '👍👍', '👍👍🏻', '👍👍👍', '👍👍👍👍', '👍👍👍👍👍', '👍👍👍👍👍👍👍👍👍', '👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍❤️❤️❤️', '👍👏👏', '👍😋', '👍😍', '👍😍😍'

In [62]:
from collections import Counter

emoji_counts = Counter(all_emojis)
for emoji, count in emoji_counts.most_common(20):
    print(emoji, count)

😍 212
❤️ 100
👍 99
🥰 94
👍🏻 91
🏻 84
☺️ 45
🔥 41
😊 39
😍😍😍 32
😋 29
♀️ 28
🤍 23
😘 21
✨ 21
😻 18
👌 17
💕 15
🤤 14
🔥🔥🔥 13


In [ ]:
Для эмодзи отдельная колонка с транскрипцией, ЕСЛИ ВОТ ТАКОЙ ТЕКСТ то не превращаем в нижний регистр

In [96]:
import emoji

str_emojis = ""
def extract_emojis(text):
    emojis = []
    global str_emojis 
    data = emoji.emoji_list(text)
    for item in data:
        emojis.append(item['emoji'])
        str_emojis += item['emoji']
    if not emojis:
        return 'None'
    else:
        return emojis

test = "Очень вкусно 😍👍🏻 но дорого 😢🤦🏻‍♀️"
print(test)
print(extract_emojis(test))

Очень вкусно 😍👍🏻 но дорого 😢🤦🏻‍♀️
['😍', '👍🏻', '😢', '🤦🏻\u200d♀️']


In [97]:
str_emojis

'😍👍🏻😢🤦🏻\u200d♀️'

In [99]:
emojis_unique = set(str_emojis)
str_emojis_unique = ""
for i in emojis_unique:
    str_emojis_unique += i




In [100]:
str_emojis_unique

'😘🙄✨❗🤞🥞🧿😐\u200d💔👋🌹😩👎🧔😇😁🤣👍💌🤭🏾😊🫠🙈🤗😂😻💜😍♥✌🫂🥀😆🔝🎊✅🥵💙↕\U0001fa76🔥💃🧘🍾🩹💫🙃😔💚✊😃🫵😋😒😜️🍴🫱🤢🥹💥🥲🍀💞💪😬🤙🤪🥺🤌😢🫲💨💓🫶🤏🌄🥰🏼💀😾💕🍣\U0001fa75🤝😹🫰👏💗💛😉\U0001fa77😄🚨🤨🎉🤮♂😤🤑💖🙏🧚⚡♀🤤🤯❤😀🥂😑😭🤬🙌🏽☺🤍🤩🤔💋👌🌺🌸💯☀🍤🏿🥱🥘💐🎵🏻🤟🎄😎😮🚬😓🙂☕🤲😫🍕🥴🥳🤷🤦🧡😞💘😡🍮☹😅🌟❣🫡🍳😌🥩'

In [95]:
type(str_emojis_unique)

str

In [98]:
df['emojis'] = df['text'].apply(lambda x: extract_emojis(x))

In [74]:
df

,author,author_reviews_count,rating,text,text_length,date,sentiment,text_1,emojis
0,Арайлым Куандык,2,5,От всей души благодарим официантку Гулжайну за...,150,2025-10-05,positive,От всей души благодарим официантку Гулжайну за...,[❤️‍🔥]
1,Merey ♥️,16,1,Обслуживание оставляет желать лучшего. Персона...,100,2025-10-05,negative,Обслуживание оставляет желать лучшего. Персона...,None
2,Adina Nursapina,1,5,"очень милый персонал,осталась очень довольной😍😍😍",48,2025-10-03,positive,"очень милый персонал,осталась очень довольной😍😍😍","[😍, 😍, 😍]"
3,Zhigerkizi demi,41,5,Реально вкусно и суперские милкшейки,36,2025-10-03,positive,Реально вкусно и суперские милкшейки,None
4,Михаил Класс,21,5,"Очень красивое и вкусное место, идеально подхо...",225,2025-10-03,positive,"Очень красивое и вкусное место, идеально подхо...",None
...,...,...,...,...,...,...,...,...,...
2307,Айтолкын Жанкелді,15,1,"Вчера были у вас, интерьер все круто. Но обслу...",371,2021-12-28,negative,"Вчера были у вас, интерьер все круто. Но обслу...",None
2308,Артур Артур,14,1,"Пришли компанией в полупустой зал, ожидали око...",236,2021-12-26,negative,"Пришли компанией в полупустой зал, ожидали око...","[😂, 🧔🏻‍♂️, ✌🏻]"
2309,Карина Сапашева,4,2,"Были здесь вчера с подругами, впечатления оста...",386,2021-12-26,negative,"Были здесь вчера с подругами, впечатления оста...",None
2310,Sary Qyz,64,5,5 поставлю за вкусную кухню👍🏻обслуживание было...,169,2021-12-24,positive,5 поставлю за вкусную кухню👍🏻обслуживание было...,"[👍🏻, 😁, 😄]"


In [75]:
all_emojis_clean = []

for emoji_list in df['emojis'].dropna():
    all_emojis_clean.extend(emoji_list)

emoji_counts = Counter(all_emojis_clean)

print(len(emoji_counts))
for em, count in emoji_counts.most_common(30):
    print(em, count)

191
N 1098
o 1098
n 1098
e 1098
😍 459
👍 237
❤️ 212
👍🏻 178
🥰 175
🔥 110
🫶🏻 87
🤍 63
☺️ 58
😊 52
😋 49
😘 40
✨ 38
🫶 35
👍🏼 33
😻 31
🤩 31
🤤 29
😂 28
🤗 28
👌 23
🙌🏻 23
💯 23
👎 23
💕 22
🥹 21


In [103]:
sample_texts = df['text_1']


unusual_chars = set()
for text in sample_texts:
    for char in text:
        if char in emoji.EMOJI_DATA:
            continue
        if not (char.isalnum() or char.isspace() or char in ',.!?:;()"-\'' or char in str_emojis_unique):
            unusual_chars.add(char)

print(sorted(unusual_chars))

['%', '&', '*', '+', '/', '>', '@', '[', ']', '_', '«', '»', '–', '—', '‘', '’', '“', '”', '…', '₸']


20

In [120]:
# Ваш список символов
symbols_to_find = ['%', '&', '*', '+', '/', '>', '@', '[', ']', '_', '«', '»', '–', '—', '‘', '’', '“', '”', '…', '₸']

# Создание паттерна для регулярного выражения, экранируя специальные символы
# Специальные символы в вашем списке: *, +, /, [, ]
pattern = '|'.join([
    r'\%', r'&', r'\*', r'\+', r'\/', r'>', r'@', r'\[', r'\]', r'_', 
    r'«', r'»', r'–', r'—', r'‘', r'’', r'“', r'”', r'…', r'₸'
])



# Вывод паттерна для проверки (необязательно)
# print(f"Используемый паттерн: {pattern}") 
# Используемый паттерн: \%|&|\*|\+|\/|>|@|\[|\]|_|«|»|–|—|‘|’|“|”|…|₸

# Фильтрация DataFrame
with_all_symbols = df[df['text_1'].str.contains(pattern, na=False)]

# Вывод результатов
print(f"Отзывов с любым из символов: {len(with_all_symbols)}")
print("\nПримеры:")
print(with_all_symbols['text_1'].head(5).tolist())

Отзывов с любым из символов: 3

Примеры:
['Забыла спросить имя официанта, обслуживание 10/10. Спасибо большое:>', 'Мы с подругами сегодня первый раз посетили Love_coffeе. И нам все понравилось.Персоналы внимательные! Нас обслуживали Али и Шынар. Спасибо большое за обслуживание! Спасибо! Обязательно вернемся!!!', 'Офф очень красивая и добрая, но обслуживание минус))(:);&(@;&:&: 🤢😉🚬🫶']


In [119]:
print(with_all_symbols['text_1'].head(10).tolist())

['Мы с подругами сегодня первый раз посетили Love_coffeе. И нам все понравилось.Персоналы внимательные! Нас обслуживали Али и Шынар. Спасибо большое за обслуживание! Спасибо! Обязательно вернемся!!!', 'Бар и кухня нифиговые, обслуживание ахеринн*е , Томирис молодец прям молодец Томирис классная такая молодец Томирис самый лучший офик', '14 февраля заранее с*ка написал спросил можно ли забронировать место вечером. Ответили что после 18:00 брони нет, принимают по факту визита. Ок, приехали к ним с жопы мира в 18:30, половина столов пустые, садимся за ближайший, на что нам говорят что стол свободен до 19:30 ибо его забронировали. Спрашиваю какого *уя он забронирован если вы сказали по факту вечером гостей принимаете ? В ответ было: забыли сказать что столики еще вчера все забронировали. И в добавок еще сказали что действует обязательный депозит 7000тг с человека. Вы че о*уели совсем?! Нас было 2 человека, мы должны заплатить 14тыс и при этом меньше чем за час успеть поесть? Могли бы все э

In [121]:
import pandas as pd
# Предполагаем, что ваш DataFrame называется df, а столбец с текстом — 'text_clean'.

# Список символов, которые вы хотите найти
symbols_to_find = ['%', '&', '*', '+', '/', '>', '@', '[', ']', '_', '«', '»', '–', '—', '‘', '’', '“', '”', '…', '₸']

print("=== Поиск примеров по каждому символу ===")
print("-" * 45)

# Проходим по каждому символу в списке
for symbol in symbols_to_find:
    # Важно: для специальных символов регулярных выражений нужно использовать r'\символ'
    # Например, для '*' нужно r'\*'
    
    # Экранируем символ, если он имеет специальное значение в регулярных выражениях (regex)
    # Используем re.escape для надежного экранирования
    import re
    
    # Паттерн для поиска одного конкретного символа
    # re.escape() экранирует все специальные символы, гарантируя, что мы ищем сам символ
    pattern = re.escape(symbol)
    
    # Фильтруем DataFrame по наличию символа
    # na=False исключает пропуски (NaN)
    with_symbol = df[df['text_1'].str.contains(pattern, na=False)]
    
    count = len(with_symbol)
    
    print(f"Символ '{symbol}': Найдено {count} отзывов")
    
    # Выводим пример, если отзывы найдены
    if count > 0:
        # .iloc[0] берет первый элемент, .tolist() преобразует его в список
        example = with_symbol['text_1'].iloc[0]
        print(f"  Пример: {example}")
    else:
        print(f"  Пример: Не найдено.")
        
    print("-" * 45)

=== Поиск примеров по каждому символу ===
---------------------------------------------
Символ '%': Найдено 42 отзывов
  Пример: Туған күн иесіне -10% деп еді, істемедіі 🙄🙄🙄 а так бары зоор 😋
---------------------------------------------
Символ '&': Найдено 1 отзывов
  Пример: Офф очень красивая и добрая, но обслуживание минус))(:);&(@;&:&: 🤢😉🚬🫶
---------------------------------------------
Символ '*': Найдено 4 отзывов
  Пример: Бар и кухня нифиговые, обслуживание ахеринн*е , Томирис молодец прям молодец Томирис классная такая молодец Томирис самый лучший офик
---------------------------------------------
Символ '+': Найдено 19 отзывов
  Пример: было вкусно, официант тоже хорошо обслуживал. красивый интерьер ,уютно +дали милые одеяло . очень понравилось , особенно обслуживание ❤️
---------------------------------------------
Символ '/': Найдено 91 отзывов
  Пример: очень остались довольны этим заведением, Администраторша очень комфортная, у нас были именинницы сестренки,Сотрудники пом

In [124]:
def normalize_text(text):
    # 1. Типографские кавычки -> обычные
    text = text.replace('«', '"').replace('»', '"')
    text = text.replace('"', '"').replace('"', '"')
    text = text.replace(''', "'").replace(''', "'")
    
    # 2. Тире -> дефис
    text = text.replace('–', '-').replace('—', '-')
    
    # 3. Многоточие
    text = text.replace('…', '...')

    return text

test_idx = df[df['text_1'].str.contains('«|»', na=False)].index[6]
before = df.loc[test_idx, 'text_1']
after = normalize_text(before)

print(before[:200])
print(after[:200])

«Это место — рай для любителей кофе! Каждый глоток — это удовольствие, которое раскрывает глубину вкуса и ароматов. Бариста — настоящие мастера своего дела, они не только готовят идеальный кофе, но и 
"Это место - рай для любителей кофе! Каждый глоток - это удовольствие, которое раскрывает глубину вкуса и ароматов. Бариста - настоящие мастера своего дела, они не только готовят идеальный кофе, но и 


In [125]:
df['text_normalized'] = df['text_1'].apply(normalize_text)

In [129]:
df

,author,author_reviews_count,rating,text,text_length,date,sentiment,text_1,emojis,text_normalized
0,Арайлым Куандык,2,5,От всей души благодарим официантку Гулжайну за...,150,2025-10-05,positive,От всей души благодарим официантку Гулжайну за...,[❤️‍🔥],От всей души благодарим официантку Гулжайну за...
1,Merey ♥️,16,1,Обслуживание оставляет желать лучшего. Персона...,100,2025-10-05,negative,Обслуживание оставляет желать лучшего. Персона...,None,Обслуживание оставляет желать лучшего. Персона...
2,Adina Nursapina,1,5,"очень милый персонал,осталась очень довольной😍😍😍",48,2025-10-03,positive,"очень милый персонал,осталась очень довольной😍😍😍","[😍, 😍, 😍]","очень милый персонал,осталась очень довольной😍😍😍"
3,Zhigerkizi demi,41,5,Реально вкусно и суперские милкшейки,36,2025-10-03,positive,Реально вкусно и суперские милкшейки,None,Реально вкусно и суперские милкшейки
4,Михаил Класс,21,5,"Очень красивое и вкусное место, идеально подхо...",225,2025-10-03,positive,"Очень красивое и вкусное место, идеально подхо...",None,"Очень красивое и вкусное место, идеально подхо..."
...,...,...,...,...,...,...,...,...,...,...
2307,Айтолкын Жанкелді,15,1,"Вчера были у вас, интерьер все круто. Но обслу...",371,2021-12-28,negative,"Вчера были у вас, интерьер все круто. Но обслу...",None,"Вчера были у вас, интерьер все круто. Но обслу..."
2308,Артур Артур,14,1,"Пришли компанией в полупустой зал, ожидали око...",236,2021-12-26,negative,"Пришли компанией в полупустой зал, ожидали око...","[😂, 🧔🏻‍♂️, ✌🏻]","Пришли компанией в полупустой зал, ожидали око..."
2309,Карина Сапашева,4,2,"Были здесь вчера с подругами, впечатления оста...",386,2021-12-26,negative,"Были здесь вчера с подругами, впечатления оста...",None,"Были здесь вчера с подругами, впечатления оста..."
2310,Sary Qyz,64,5,5 поставлю за вкусную кухню👍🏻обслуживание было...,169,2021-12-24,positive,5 поставлю за вкусную кухню👍🏻обслуживание было...,"[👍🏻, 😁, 😄]",5 поставлю за вкусную кухню👍🏻обслуживание было...


In [131]:
duplicates = df['text_normalized'].duplicated(keep='first')
print(duplicates.sum())

if duplicates.sum() > 0:
    dup_texts = df[duplicates]['text_normalized'].head(3)

    for i, text in enumerate(dup_texts, 1):
        print(i, text[:150])

2
1 лучших кафе в котором был, у повора просто золотые руки 😘
2 Ждали 1,5 часа свой заказ, официант еле как подошел принял заказ, в итоге не дождались. Все столики которые пришли вместе с нами уже покушали и уходил


In [134]:
has_url = df['text_normalized'].str.contains(r'http|www\.|\.[a-z]{2,3}/', case=False, regex=True, na=False)
print(has_url.sum())

has_phone = df['text_normalized'].str.contains(r'\+7|8\s?\(\d{3}\)?|\d{3}[-\s]?\d{2}', regex=True, na=False)
print(has_phone.sum())

if has_phone.sum() > 0:
    print(df[has_phone]['text_normalized'].tolist())

0
5
['Обслуживание -0 , не советую этот заведение если вы хотите спокойно кушать , провести время хорошо без неприятности тогда попробуйте другие варианты, -0000000', 'Пришли компанией 15 июня в 10 утра 1. приборы грязные (начиная ножом заканчивая бокалом) 2. попросили принести салфеток так как за столом их не было ( пришлось брать самому с другого столика) 3. Заказали 2 чая (Ташкентский и эрэлгрей с малиной) 3.1 с пеною у рта доказывали что в Ташкентский чай не идёт нават (два раза подходили и утверждали что не идёт) 3.2 эрел грей не принесли а вместо него принесли чай с кучай лимона, пошел сам разбираться 4. Зовешь персонал ни кто не подходит, все время сами за ними ходили и по итогу 12% обслуживание, здорово, сам ходишь их ищешь и плати им 5. Ждали заказ 40 минут Впечатление на счет этого заведения остались паршивыми Если интересно кто написал и хотите честный отзыв обращайтесь 8 7778288663 Владислав', 'өте әдемі жер,тамақтары да өте дәмді😊💗іші де керемет,атмосферасы да кайфовый,кел

In [139]:
suspicious_repeats = df['text_normalized'].str.contains(r'(.)\1{4,}', regex=True, na=False)
print(suspicious_repeats.sum())
print(df[suspicious_repeats]['text_normalized'].head(5).tolist())

suspicious_ratings = df['text_normalized'].str.contains(r'\d{4,}/\d+|\d{3,}%', regex=True, na=False)
print(suspicious_ratings.sum())
print(df[suspicious_ratings]['text_normalized'].head(5).tolist())

short_positive = (df['text_length'] < 50) & (df['rating'] == 5)
print(short_positive.sum())

print(df[short_positive]['text_normalized'].head(5).tolist())

36
['👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍❤️❤️❤️', 'Обслуживание -0 , не советую этот заведение если вы хотите спокойно кушать , провести время хорошо без неприятности тогда попробуйте другие варианты, -0000000', 'Хостес АЛИНА , ДАЯШЫ АЛМАС 100/ден 100 сондай ақ көңіл жүздері жылы қуанып қарсы алғандарына қарап келгеніңе қуанасың 😍😍😍😍😍Тамақтары дәмді шайлары қою ♥️ең сүйікті Лова ға айналды', 'такое классное местооооо😻😻😻Интерьер стильный, персонал вежливый', 'Испортили все настроение! Только прилетела в Алматы, встретилась с дочерью. Хотелось вкусно поужинать и побеседовать. Но...... Нас будто не видели. Сами сходили за меню, и опять еле дождались официантку. Уточнили состав блюд, которые она не знала, и определились с выбором. Принесли одно блюдо, (кстати то, которое мы уточняли по составу, но не заказывали). Настроение портить до конца не хотелось, поэтому не стали спорить, тем более очень хотелось есть. Принесли дочери заказанное, но напитка так и нет. Парень официант несколько раз заносил и в

/tmp/ipykernel_335832/950630815.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  suspicious_repeats = df['text_normalized'].str.contains(r'(.)\1{4,}', regex=True, na=False)


In [145]:
has_caps = df['text_normalized'].str.contains(r'[А-ЯЁ]{5,}', regex=True, na=False)
print(has_caps.sum())

if has_caps.sum() > 0:
    print(df[has_caps]['text_normalized'].head(3).tolist())

has_numbers = df['text_normalized'].str.contains(r'\d', na=False)
print(has_numbers.sum(), has_numbers.sum()/len(df)*100)
print(df[has_numbers]['text_normalized'].tail().tolist())

34
['Хостес АЛИНА , ДАЯШЫ АЛМАС 100/ден 100 сондай ақ көңіл жүздері жылы қуанып қарсы алғандарына қарап келгеніңе қуанасың 😍😍😍😍😍Тамақтары дәмді шайлары қою ♥️ең сүйікті Лова ға айналды', 'Это худший опыт в заведении за долгое время! Заведение очень уютное и красивое, интерьер действительно радует глаз. Но, к сожалению, на этом плюсы заканчиваются. Рамен ждали 50 минут, роллы - полтора часа, и только спустя всё это время нам сообщили, что суши вообще нет в наличии. Обслуживание - полный ноль, никакого внимания к гостям. Потратили кучу времени впустую.😔 Пришли после работы просто вкусно покушать, ноооо увыыыы. НЕ СОВЕТУЮ', 'Испортили все настроение! Только прилетела в Алматы, встретилась с дочерью. Хотелось вкусно поужинать и побеседовать. Но...... Нас будто не видели. Сами сходили за меню, и опять еле дождались официантку. Уточнили состав блюд, которые она не знала, и определились с выбором. Принесли одно блюдо, (кстати то, которое мы уточняли по составу, но не заказывали). Настроение п

In [147]:
import re

def detect_languages(text):
    has_russian = bool(re.search(r'[а-яёА-ЯЁ]', text))
    has_kazakh = bool(re.search(r'[әіңғүұқөһӘІҢҒҮҰҚӨҺ]', text))
    has_english = bool(re.search(r'[a-zA-Z]{3,}', text))

    langs = []
    if has_russian: langs.append('ru')
    if has_kazakh: langs.append('kz')
    if has_english: langs.append('en')

    return ','.join(langs) if langs else 'unknown'

df['languages'] = df['text_normalized'].apply(detect_languages)

print(df['languages'].value_counts())

languages
ru          2006
ru,kz        226
ru,en         67
en             7
ru,kz,en       5
unknown        1
Name: count, dtype: int64


In [151]:
df[df['languages'] == 'kz']

,author,author_reviews_count,rating,text,text_length,date,sentiment,text_1,emojis,text_normalized,languages


In [152]:
df.to_csv('reviews_cleaned_step1.csv', index=False)
print("Сохранено в reviews_cleaned_step1.csv")
print(f"\nВсего строк: {len(df)}")
print(f"Столбцов: {len(df.columns)}")
print("\nНовые столбцы:")
print([col for col in df.columns if col not in ['author', 'author_reviews_count', 'rating', 'text', 'text_length', 'date', 'sentiment']])

Сохранено в reviews_cleaned_step1.csv

Всего строк: 2312
Столбцов: 11

Новые столбцы:
['text_1', 'emojis', 'text_normalized', 'languages']


# second step


In [155]:
import razdel 

test_text = "Очень вкусно! Цены от 1000₸. Официант Алмас - молодец👍"
tokens = [token.text for token in razdel.tokenize(test_text)]
print(test_text)
print(tokens)

Очень вкусно! Цены от 1000₸. Официант Алмас - молодец👍
['Очень', 'вкусно', '!', 'Цены', 'от', '1000', '₸', '.', 'Официант', 'Алмас', '-', 'молодец👍']


In [157]:
test2 = "Кафе 'Лова' - 10/10, рекомендую!!!"
tokens2 = [token.text for token in razdel.tokenize(test2)]
print(test2)
print(tokens2)

Кафе 'Лова' - 10/10, рекомендую!!!
['Кафе', "'", 'Лова', "'", '-', '10/10', ',', 'рекомендую', '!!!']


In [159]:
def tokenize_text(text, remove_punct=True, remove_numbers=True):
    tokens = [token.text for token in razdel.tokenize(text)]

    if remove_punct:
        tokens = [t for t in tokens if t not in string.punctuation and t not in '—–…«»""''']

    if remove_numbers:
        tokens = [t for t in tokens if not t.isdigit()]

    return tokens

test = "Кафе 'Лова' - 10/10, цены от 1000₸. Рекомендую!!!"
print(test)
print(tokenize_text(test, remove_punct=False, remove_numbers=False))
print(tokenize_text(test, remove_punct=True, remove_numbers=False))
print(tokenize_text(test, remove_punct=True, remove_numbers=True))

Кафе 'Лова' - 10/10, цены от 1000₸. Рекомендую!!!
['Кафе', "'", 'Лова', "'", '-', '10/10', ',', 'цены', 'от', '1000', '₸', '.', 'Рекомендую', '!!!']
['Кафе', 'Лова', '10/10', 'цены', 'от', '1000', '₸', 'Рекомендую', '!!!']
['Кафе', 'Лова', '10/10', 'цены', 'от', '₸', 'Рекомендую', '!!!']


In [167]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

test_words = ['вкусные', 'понравилось', 'официанты', 'обслуживание', 'цены']
for word in test_words:
    lemma = morph.parse(word)[0].normal_form
    print(word, lemma)
    

вкусные вкусный
понравилось понравиться
официанты официант
обслуживание обслуживание
цены цена


In [198]:
def preprocess_text(text, languages, remove_punct=True, remove_numbers=False, lemmatize=True):
    if 'kz' in languages:
        return [], ''
    tokens = [token.text for token in razdel.tokenize(text)]

    if remove_punct:
        tokens = [t for t in tokens if t not in string.punctuation and t not in '—–…«»""''']
    if remove_numbers:
        tokens = [t for t in tokens if not t.isdigit()]
    if lemmatize:
        tokens = [morph.parse(t)[0].normal_form for t in tokens]

    return tokens, ' '.join(tokens)

# test_review = df['text_normalized'].iloc[0]
# print(test_review[:150])
# processed = preprocess_text(test_review, remove_numbers=False)
# print(processed[:20])

In [183]:
import time 

start = time.time()
sample = df['text_normalized'].head(100)
sample_processed = sample.apply(lambda x: preprocess_text(x, remove_numbers=False))\
# elapsed = time.time() - start
elapsed = time.time() - start
print(f"Обработано 100 отзывов за {elapsed:.2f} секунд")
print(f"Скорость: {100/elapsed:.1f} отзывов/сек")
print(f"Примерное время для всех {len(df)} отзывов: {(len(df)*elapsed/100)/60:.1f} минут")

for i in range(3):
    print(f"\n{i+1}. {' '.join(sample_processed.iloc[i][:15])}...")

Обработано 100 отзывов за 0.26 секунд
Скорость: 379.1 отзывов/сек
Примерное время для всех 2312 отзывов: 0.1 минут

1. от весь душа благодарить официантка гулжайн за прекрасный обслуживание всё быть очень душевно с улыбка...

2. обслуживание оставлять желать хороший персонал не проявить внимательность и вежливость не рекомендовать...

3. очень милый персонал остаться очень довольной😍😍😍...


In [201]:
result = df.apply(lambda x: preprocess_text(x['text_normalized'], x['languages']), axis=1)
df['tokens'] = result.apply(lambda x: x[0])
df['text_lemmatized'] = result.apply(lambda x: x[1])
print(df.loc[10, 'text_normalized'][:150])
print(df.loc[10, 'text_lemmatized'][:150])

Қайырлы күн! кеше сіздерде туған күн тойладық. Бізге қатты ұнады. Тағамдар сондай дәмді болды, әсіресе стейк керемет пісірілген. Бізге қызмет көрсетке



In [205]:
print("\nПример русского (обработан):")
ru_idx = df[df['languages'] == 'en'].index[0]
print(df.loc[ru_idx, 'text_lemmatized'][:150])


Пример русского (обработан):
wonderful place to come with friends and family lovely atmosphere and wonderful service food is delicious thank you to bota who made sure we had a gre


In [296]:
# import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk.tokenize import word_tokenize

# # Скачаем необходимые данные (один раз)
# try:
#   nltk.data.find('tokenizers/punkt')
# except LookupError:
#   nltk.download('punkt')
#   nltk.download('wordnet')
#   nltk.download('omw-1.4')

# Создаем лемматизатор для английского
en_lemmatizer = WordNetLemmatizer()

def preprocess_text(text, text_type, remove_punct=True, remove_numbers=False, lemmatize=True):
  """
  Обработка текста с учетом языка
  text_type: 'russian', 'english', 'kazakh', 'mixed_*', 'translit', etc.
  """

  # Пропускаем казахский и транслит
  if text_type in ['kazakh', 'mixed_kk_ru', 'translit', 'latin_unknown', 'no_letters']:
      return [], ''

  # Токенизация
  if text_type == 'english':
      # Английская токенизация
      tokens = word_tokenize(text.lower())
  else:
      # Русская токенизация (russian, mixed_ru_en)
      tokens = [token.text for token in razdel.tokenize(text)]

  # Убираем пунктуацию
  if remove_punct:
      tokens = [t for t in tokens if t not in string.punctuation and t not in '—–…«»""''']

  # Убираем числа
  if remove_numbers:
      tokens = [t for t in tokens if not t.isdigit()]

  # Лемматизация
  if lemmatize:
      if text_type == 'english':
          tokens = [en_lemmatizer.lemmatize(t) for t in tokens]
      else:
          # Русский (или mixed)
          tokens = [morph.parse(t)[0].normal_form for t in tokens]

  return tokens, ' '.join(tokens)

# Тестируем на разных языках
print("Тестирование:")
print("\n1. Русский:")
ru_text = "Очень вкусные блюда и отличное обслуживание"
tokens, lemma = preprocess_text(ru_text, 'russian')
print(f"Исходный: {ru_text}")
print(f"Обработан: {lemma}")

print("\n2. Английский:")
en_text = "Wonderful dishes and excellent service"
tokens, lemma = preprocess_text(en_text, 'english')
print(f"Исходный: {en_text}")
print(f"Обработан: {lemma}")

Тестирование:

1. Русский:
Исходный: Очень вкусные блюда и отличное обслуживание
Обработан: очень вкусный блюдо и отличный обслуживание

2. Английский:
Исходный: Wonderful dishes and excellent service
Обработан: wonderful dish and excellent service


In [215]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [298]:
print("Обработка всех отзывов...")
import time

start = time.time()

# Применяем функцию с учетом text_type
result = df.apply(lambda row: preprocess_text(row['text_normalized'], row['text_type']), axis=1)

df['tokens'] = result.apply(lambda x: x[0])
df['text_lemmatized'] = result.apply(lambda x: x[1])

elapsed = time.time() - start
print(f"Готово за {elapsed:.2f} секунд!")

# Статистика обработки
processed = df['text_lemmatized'].str.len() > 0
print(f"\nОбработано отзывов: {processed.sum()} из {len(df)}")
print(f"Пропущено (казахский/транслит): {(~processed).sum()}")

# Примеры
print("\n=== Примеры обработки ===")
print("\n1. Русский:")
ru_idx = df[df['text_type'] == 'russian'].index[0]
print(f"Исходный: {df.loc[ru_idx, 'text_normalized'][:100]}")
print(f"Обработан: {df.loc[ru_idx, 'text_lemmatized'][:100]}")

print("\n2. Английский:")
en_idx = df[df['text_type'] == 'english'].index[0]
print(f"Исходный: {df.loc[en_idx, 'text_normalized'][:100]}")
print(f"Обработан: {df.loc[en_idx, 'text_lemmatized'][:100]}")

print("\n3. Казахский (пропущен):")
kk_idx = df[df['text_type'] == 'kazakh'].index[0]
print(f"Исходный: {df.loc[kk_idx, 'text_normalized'][:100]}")
print(f"Обработан: '{df.loc[kk_idx, 'text_lemmatized']}'")

Обработка всех отзывов...
Готово за 6.01 секунд!

Обработано отзывов: 2312 из 2312
Пропущено (казахский/транслит): 0

=== Примеры обработки ===

1. Русский:
Исходный: От всей души благодарим официантку Гулжайну за прекрасное обслуживание.Всё было очень душевно, с улы
Обработан: от весь душа благодарить официантка гулжайн за прекрасный обслуживание.всё быть очень душевно с улыб

2. Английский:
Исходный: Wonderful place to come with friends and family. Lovely atmosphere and wonderful service. Food is de
Обработан: wonderful place to come with friend and family lovely atmosphere and wonderful service food is delic

3. Казахский (пропущен):


IndexError: index 0 is out of bounds for axis 0 with size 0

In [230]:
df[df['text_type'] == 'no_letters']

,author,author_reviews_count,rating,text,text_length,date,sentiment,text_1,emojis,text_normalized,languages,tokens,text_lemmatized,text_type
64,Дана Казакбаева,1,5,👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍❤️❤️❤️,32,2025-08-26,positive,👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍❤️❤️❤️,"[👍, 👍, 👍, 👍, 👍, 👍, 👍, 👍, 👍, 👍, 👍, 👍, 👍, 👍, 👍, ...",👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍👍❤️❤️❤️,unknown,[],,no_letters


In [208]:
from langdetect import detect, LangDetectException

def detect_language(text):
    try:
        lang = detect(text)
        return lang
    except LangDetectException:
        return 'unknown'
test_texts = [
    "Wonderful place to come with friends",
    "Kalyan top krepki umeuyt zabivat",
    "Очень вкусно и атмосферно",
    "Өте дәмді тамақ",
]
for text in test_texts:
    lang = detect_language(text)
    print(lang, text)

en Wonderful place to come with friends
hr Kalyan top krepki umeuyt zabivat
ru Очень вкусно и атмосферно
uk Өте дәмді тамақ


In [209]:
def classify_text_type(text):
      """Классифицирует тип текста для последующей обработки"""

      # Проверяем специфические буквы
      kazakh_letters = set('әіңғүұқөһӘІҢҒҮҰҚӨҺ')
      russian_letters = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ')

      has_kazakh = any(letter in kazakh_letters for letter in text)
      has_russian = any(letter in russian_letters for letter in text)
      has_latin = bool(re.search(r'[a-zA-Z]{3,}', text))  # минимум 3 латинские буквы подряд

      # Определяем тип
      if has_kazakh and has_russian:
          return 'mixed_kk_ru'
      elif has_kazakh:
          return 'kazakh'
      elif has_russian and has_latin:
          return 'mixed_ru_en'
      elif has_russian:
          return 'russian'
      elif has_latin:
          # Пытаемся определить - английский или транслит
          try:
              detected = detect(text)
              if detected == 'en':
                  return 'english'
              else:
                  return 'translit'  # вероятно транслит
          except:
              return 'latin_unknown'
      else:
          return 'no_letters'

# Применяем классификацию
df['text_type'] = df['text_normalized'].apply(classify_text_type)

# Статистика
print("Распределение по типам текста:")
print(df['text_type'].value_counts())
print(f"\nВсего: {len(df)}")

Распределение по типам текста:
text_type
russian        2006
mixed_kk_ru     231
mixed_ru_en      67
english           6
no_letters        1
translit          1
Name: count, dtype: int64

Всего: 2312


In [213]:
df[df['text_type'] == 'english']

,author,author_reviews_count,rating,text,text_length,date,sentiment,text_1,emojis,text_normalized,languages,tokens,text_lemmatized,text_type
513,Sandy Zh,16,5,Wonderful place to come with friends and famil...,237,2024-12-24,positive,Wonderful place to come with friends and famil...,None,Wonderful place to come with friends and famil...,en,"[wonderful, place, to, come, with, friends, an...",wonderful place to come with friends and famil...,english
591,Klara Bakyt,1,5,"The environment is very good, the service atti...",97,2024-12-12,positive,"The environment is very good, the service atti...",None,"The environment is very good, the service atti...",en,"[the, environment, is, very, good, the, servic...",the environment is very good the service attit...,english
850,Lana Kennedy,13,1,Terrible place. Unsanitary conditions.,38,2024-09-28,negative,Terrible place. Unsanitary conditions.,None,Terrible place. Unsanitary conditions.,en,"[terrible, place, unsanitary, conditions]",terrible place unsanitary conditions,english
1082,Glebbbbbbbb Bbbbbbbbelg,3,5,Leila made sure that we have everything we nee...,104,2024-07-20,positive,Leila made sure that we have everything we nee...,None,Leila made sure that we have everything we nee...,en,"[leila, made, sure, that, we, have, everything...",leila made sure that we have everything we nee...,english
1396,Claire Sutton,1,1,I have just tried this restaurant and was very...,681,2024-03-31,negative,I have just tried this restaurant and was very...,None,I have just tried this restaurant and was very...,en,"[i, have, just, tried, this, restaurant, and, ...",i have just tried this restaurant and was very...,english
2234,🇨🇦Najib MEKKI🇵🇸🇲🇦,3,1,The waitress took away my full glass of lemona...,390,2022-05-26,negative,The waitress took away my full glass of lemona...,None,The waitress took away my full glass of lemona...,en,"[the, waitress, took, away, my, full, glass, o...",the waitress took away my full glass of lemona...,english


Отлично! Давай подведем итоги и определим следующие шаги.

  📊 Что уже сделано:

  ✅ Базовая чистка:
  - Убрали переносы строк
  - Нормализовали типографские символы
  - Извлекли эмодзи в отдельную колонку

  ✅ Токенизация и лемматизация:
  - Обработали русские тексты (pymorphy2)
  - Обработали английские тексты (nltk)
  - Пометили казахские/транслит для будущей обработки

  ✅ Классификация текстов:
  - Определили типы текстов (russian, english, kazakh, mixed, translit)

  ---
  🎯 Что дальше делать:

  Шаг 23: Удаление стоп-слов
  # Убираем частые незначимые слова (и, в, на, the, is, etc.)

  Шаг 24: Базовая статистика для анализа
  # Частота слов, биграммы, триграммы
  # TF-IDF для важных слов

  Шаг 25: Подготовка к моделям
  # Векторизация текстов
  # Train/test split

  ---
  С чего начнем? Предлагаю со стоп-слов - это быстро и улучшит качество данных для моделей.

  Продолжаем?

In [231]:
from nltk.corpus import stopwords

try:
    stopwords.words('russian')
except LookupError:
    nltk.download('stopwords')

russian_stopwords = set(stopwords.words('russian'))
english_stopwords = set(stopwords.words('english'))

print(len(russian_stopwords))
print(len(english_stopwords))

print(list(russian_stopwords)[:20])
print(list(english_stopwords)[:20])

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...


151
198
['при', 'моя', 'они', 'всегда', 'много', 'ну', 'того', 'них', 'или', 'себя', 'хоть', 'более', 'вот', 'себе', 'к', 'им', 'всех', 'куда', 'по', 'ней']
['off', 'it', 'mustn', "i'm", 'she', 'which', 'where', 'an', 'does', 'their', "doesn't", "needn't", "mightn't", "shan't", "you'll", "it's", 'few', 'during', 'here', "wouldn't"]


[nltk_data]   Unzipping corpora/stopwords.zip.


In [246]:
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

all_tokens = []
for tokens_list in df['tokens'].dropna():
    if isinstance(tokens_list, list):
        all_tokens.extend(tokens_list)

print(len(all_tokens))
print(len(set(all_tokens)))

55279
5775


In [247]:
word_freq = Counter(all_tokens)
top_50 = word_freq.most_common(50)

for i, (word, count) in enumerate((top_50), 1):
    print(i, word, count)

1 и 2110
2 очень 1421
3 не 1177
4 в 898
5 быть 864
6 на 849
7 всё 794
8 обслуживание 767
9 официант 716
10 с 685
11 вкусный 609
12 мы 584
13 заведение 572
14 что 518
15 это 460
16 хороший 406
17 кухня 399
18 еда 395
19 место 391
20 но 385
21 спасибо 380
22 понравиться 350
23 за 338
24 я 333
25 приятный 324
26 вкусно 319
27 атмосфера 309
28 персонал 296
29 красивый 293
30 уютный 288
31 вежливый 280
32 блюдо 243
33 а 240
34 интерьер 231
35 как 228
36 по 221
37 так 219
38 просто 217
39 к 216
40 принести 201
41 ещё 200
42 заказ 200
43 высокий 198
44 отличный 196
45 уровень 195
46 вы 190
47 ждать 188
48 весь 187
49 большой 182
50 классный 179


In [257]:
all_stopwords = russian_stopwords | english_stopwords


stop_in_top = [(word, count) for word, count in top_50 if word.lower() in all_stopwords]

for word, count in stop_in_top:
    print(f"  {word} {count}")

print(len(stop_in_top))
significant_in_top = [(word, count) for word, count in top_50 if word.lower() not in all_stopwords]
for word, count in significant_in_top:
    print(word, count)

  и 2110
  не 1177
  в 898
  быть 864
  на 849
  с 685
  мы 584
  что 518
  но 385
  за 338
  я 333
  а 240
  как 228
  по 221
  так 219
  к 216
  вы 190
17
очень 1421
всё 794
обслуживание 767
официант 716
вкусный 609
заведение 572
это 460
хороший 406
кухня 399
еда 395
место 391
спасибо 380
понравиться 350
приятный 324
вкусно 319
атмосфера 309
персонал 296
красивый 293
уютный 288
вежливый 280
блюдо 243
интерьер 231
просто 217
принести 201
ещё 200
заказ 200
высокий 198
отличный 196
уровень 195
ждать 188
весь 187
большой 182
классный 179


In [281]:
df[df['languages'] == 'ru,kz']

,author,author_reviews_count,rating,text,text_length,date,sentiment,text_1,emojis,text_normalized,languages,tokens,text_lemmatized,text_type
5,Sailauovaaas_ _,1,4,"Туған күн иесіне -10% деп еді, істемедіі 🙄🙄🙄 а...",62,2025-09-30,positive,"Туған күн иесіне -10% деп еді, істемедіі 🙄🙄🙄 а...","[🙄, 🙄, 🙄, 😋]","Туған күн иесіне -10% деп еді, істемедіі 🙄🙄🙄 а...","ru,kz",[],,mixed_kk_ru
6,Каракат Кайраткызы,2,5,"Іші әдемі, интерьер эстетичный, тамақтары дәмд...",111,2025-09-29,positive,"Іші әдемі, интерьер эстетичный, тамақтары дәмд...",None,"Іші әдемі, интерьер эстетичный, тамақтары дәмд...","ru,kz",[],,mixed_kk_ru
10,Saule Kokanaeva,2,5,Қайырлы күн! кеше сіздерде туған күн тойладық....,206,2025-09-27,positive,Қайырлы күн! кеше сіздерде туған күн тойладық....,"[❤️, 😍, 🌺]",Қайырлы күн! кеше сіздерде туған күн тойладық....,"ru,kz",[],,mixed_kk_ru
16,Рауан Манат,2,5,Өте керемет кафе екен тағыда келуге турарлык ө...,188,2025-09-23,positive,Өте керемет кафе екен тағыда келуге турарлык ө...,None,Өте керемет кафе екен тағыда келуге турарлык ө...,"ru,kz",[],,mixed_kk_ru
61,Alibi Adilkhanuly,4,5,"Бәрі күшті, ұнады!\nПланшетпен менюден тапсыры...",66,2025-08-28,positive,"Бәрі күшті, ұнады! Планшетпен менюден тапсырыс...",[👍🏻],"Бәрі күшті, ұнады! Планшетпен менюден тапсырыс...","ru,kz",[],,mixed_kk_ru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182,Islam Bodaubek,1,5,"Рахмет сіздерге тағамдарыңыз дәмді екен, атмос...",110,2022-08-12,positive,"Рахмет сіздерге тағамдарыңыз дәмді екен, атмос...",None,"Рахмет сіздерге тағамдарыңыз дәмді екен, атмос...","ru,kz",[],,mixed_kk_ru
2238,Айтолқын Жанысбек,18,4,Тамақтардың дәмдері жақсы бірақ порциясы кішке...,135,2022-05-09,positive,Тамақтардың дәмдері жақсы бірақ порциясы кішке...,"[😊, ❤️]",Тамақтардың дәмдері жақсы бірақ порциясы кішке...,"ru,kz",[],,mixed_kk_ru
2282,Asylzat Asylzat,16,5,Тамақтары дәмді😍👍🏻 Обслуживание вообще топ👍🏻 Х...,116,2022-02-11,positive,Тамақтары дәмді😍👍🏻 Обслуживание вообще топ👍🏻 Х...,"[😍, 👍🏻, 👍🏻, 👍🏻, 😍]",Тамақтары дәмді😍👍🏻 Обслуживание вообще топ👍🏻 Х...,"ru,kz",[],,mixed_kk_ru
2290,Aidana Nurseyt,2,5,Бүгін подругаларыммен осы заведениеде болдық. ...,595,2022-02-06,positive,Бүгін подругаларыммен осы заведениеде болдық. ...,"[👍🏻, 🙂, 🙂, 🤲🏻, 🙂, 👍🏻, 🤗]",Бүгін подругаларыммен осы заведениеде болдық. ...,"ru,kz",[],,mixed_kk_ru


In [260]:
df

,author,author_reviews_count,rating,text,text_length,date,sentiment,text_1,emojis,text_normalized,languages,tokens,text_lemmatized,text_type
0,Арайлым Куандык,2,5,От всей души благодарим официантку Гулжайну за...,150,2025-10-05,positive,От всей души благодарим официантку Гулжайну за...,[❤️‍🔥],От всей души благодарим официантку Гулжайну за...,ru,"[от, весь, душа, благодарить, официантка, гулж...",от весь душа благодарить официантка гулжайн за...,russian
1,Merey ♥️,16,1,Обслуживание оставляет желать лучшего. Персона...,100,2025-10-05,negative,Обслуживание оставляет желать лучшего. Персона...,None,Обслуживание оставляет желать лучшего. Персона...,ru,"[обслуживание, оставлять, желать, хороший, пер...",обслуживание оставлять желать хороший персонал...,russian
2,Adina Nursapina,1,5,"очень милый персонал,осталась очень довольной😍😍😍",48,2025-10-03,positive,"очень милый персонал,осталась очень довольной😍😍😍","[😍, 😍, 😍]","очень милый персонал,осталась очень довольной😍😍😍",ru,"[очень, милый, персонал, остаться, очень, дово...",очень милый персонал остаться очень довольной😍😍😍,russian
3,Zhigerkizi demi,41,5,Реально вкусно и суперские милкшейки,36,2025-10-03,positive,Реально вкусно и суперские милкшейки,None,Реально вкусно и суперские милкшейки,ru,"[реально, вкусно, и, суперский, милкшейк]",реально вкусно и суперский милкшейк,russian
4,Михаил Класс,21,5,"Очень красивое и вкусное место, идеально подхо...",225,2025-10-03,positive,"Очень красивое и вкусное место, идеально подхо...",None,"Очень красивое и вкусное место, идеально подхо...","ru,en","[очень, красивый, и, вкусный, место, идеально,...",очень красивый и вкусный место идеально подход...,mixed_ru_en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2307,Айтолкын Жанкелді,15,1,"Вчера были у вас, интерьер все круто. Но обслу...",371,2021-12-28,negative,"Вчера были у вас, интерьер все круто. Но обслу...",None,"Вчера были у вас, интерьер все круто. Но обслу...",ru,"[вчера, быть, у, вы, интерьер, всё, круто, но,...",вчера быть у вы интерьер всё круто но обслужив...,russian
2308,Артур Артур,14,1,"Пришли компанией в полупустой зал, ожидали око...",236,2021-12-26,negative,"Пришли компанией в полупустой зал, ожидали око...","[😂, 🧔🏻‍♂️, ✌🏻]","Пришли компанией в полупустой зал, ожидали око...","ru,en","[прислать, компания, в, полупустой, зал, ожида...",прислать компания в полупустой зал ожидать око...,mixed_ru_en
2309,Карина Сапашева,4,2,"Были здесь вчера с подругами, впечатления оста...",386,2021-12-26,negative,"Были здесь вчера с подругами, впечатления оста...",None,"Были здесь вчера с подругами, впечатления оста...",ru,"[быть, здесь, вчера, с, подруга, впечатление, ...",быть здесь вчера с подруга впечатление остатьс...,russian
2310,Sary Qyz,64,5,5 поставлю за вкусную кухню👍🏻обслуживание было...,169,2021-12-24,positive,5 поставлю за вкусную кухню👍🏻обслуживание было...,"[👍🏻, 😁, 😄]",5 поставлю за вкусную кухню👍🏻обслуживание было...,ru,"[5, поставить, за, вкусный, кухню👍🏻обслуживани...",5 поставить за вкусный кухню👍🏻обслуживание быт...,russian


In [264]:
not_lemmatized = df[df['text_lemmatized'].isna()]
print(len(not_lemmatized))

print(not_lemmatized['text_type'].value_counts())

for idx, row in not_lemmatized.head(5).iterrows():
    print(row['text_type'])
    print(row['text_normalized'][:100])

0
Series([], Name: count, dtype: int64)


In [265]:
kk_texts = df[df['text_type'].isin(['mixed_kk_ru', 'mixed_ru_en', 'translit'])]
print(len(kk_texts))
print(kk_texts['text_type'].value_counts())

for idx, row in kk_texts.head(5).iterrows():
    print(row['text_type'])
    print(row['text_normalized'][:80])
    print(row['text_lemmatized'][:80])

299
text_type
mixed_kk_ru    231
mixed_ru_en     67
translit         1
Name: count, dtype: int64
mixed_ru_en
Очень красивое и вкусное место, идеально подходит для свиданий и приятных посиде
очень красивый и вкусный место идеально подходить для свидание и приятный посиде
mixed_kk_ru
Туған күн иесіне -10% деп еді, істемедіі 🙄🙄🙄 а так бары зоор 😋

mixed_kk_ru
Іші әдемі, интерьер эстетичный, тамақтары дәмді. но өте ұзақ уақыт күттік, қонақ

mixed_ru_en
От всей нашей компании говорим спасибо официанту - Зарине за доброжелательное об
от весь наш компания говорить спасибо официант зарина за доброжелательный обслуж
mixed_kk_ru
Қайырлы күн! кеше сіздерде туған күн тойладық. Бізге қатты ұнады. Тағамдар сонда



In [266]:
df['text_type'].unique()

array(['russian', 'mixed_ru_en', 'mixed_kk_ru', 'no_letters', 'english',
       'translit'], dtype=object)

In [273]:
import sys 
sys.path.append('work/nlp_2gis/kaznlp/kaznlp')

from kaznlp.normalization.ininorm import Normalizer
from kaznlp.tokenization.tokhmm import TokenizerHMM

kaz_normalizer = Normalizer()
kaz_tokenizer = TokenizerHMM()

def process_kazakh_text(text):
    if pd.isna(text) or text == '':
        return None

    try:
        normalized = kaz_normalizer.normalize(
            str(text),
            translit=True,
            desegmnet = 2,
            dedupe = 2,
            emojiresolve = False)
        tokens = kaz_tokenizer.tokenize(normalized)
        tokens_lower = [token.lower() for token in tokens]

        return ' '.join(tokens_lower)

    except Exception as e:
        return text.lower()

test_text = 'Өте тамаша орын! Рахмет!'
result = process_kazakh_text(test_text)

print(result)

өте тамаша орын! рахмет!


In [ ]:
from pymorphy3 import MorphAnalyzer
import razdel 

morph_ru = MorphAnalyzer()

def process_mixed_kk_ru_text(text):
    if pd.isna(text) or text == '':
        return None

    try:
        normalized = kaz_normalizer.normalize(
            str(text),
            translit=True,
            desegmnet = 2,
            dedupe = 2,
            emojiresolve = False)
        tokens = kaz_tokenizer.tokenize(normalized)
        kaz_tokens = kaz_tokenizer.tokenize(normalized)
        
        
    

In [274]:
df

,author,author_reviews_count,rating,text,text_length,date,sentiment,text_1,emojis,text_normalized,languages,tokens,text_lemmatized,text_type
0,Арайлым Куандык,2,5,От всей души благодарим официантку Гулжайну за...,150,2025-10-05,positive,От всей души благодарим официантку Гулжайну за...,[❤️‍🔥],От всей души благодарим официантку Гулжайну за...,ru,"[от, весь, душа, благодарить, официантка, гулж...",от весь душа благодарить официантка гулжайн за...,russian
1,Merey ♥️,16,1,Обслуживание оставляет желать лучшего. Персона...,100,2025-10-05,negative,Обслуживание оставляет желать лучшего. Персона...,None,Обслуживание оставляет желать лучшего. Персона...,ru,"[обслуживание, оставлять, желать, хороший, пер...",обслуживание оставлять желать хороший персонал...,russian
2,Adina Nursapina,1,5,"очень милый персонал,осталась очень довольной😍😍😍",48,2025-10-03,positive,"очень милый персонал,осталась очень довольной😍😍😍","[😍, 😍, 😍]","очень милый персонал,осталась очень довольной😍😍😍",ru,"[очень, милый, персонал, остаться, очень, дово...",очень милый персонал остаться очень довольной😍😍😍,russian
3,Zhigerkizi demi,41,5,Реально вкусно и суперские милкшейки,36,2025-10-03,positive,Реально вкусно и суперские милкшейки,None,Реально вкусно и суперские милкшейки,ru,"[реально, вкусно, и, суперский, милкшейк]",реально вкусно и суперский милкшейк,russian
4,Михаил Класс,21,5,"Очень красивое и вкусное место, идеально подхо...",225,2025-10-03,positive,"Очень красивое и вкусное место, идеально подхо...",None,"Очень красивое и вкусное место, идеально подхо...","ru,en","[очень, красивый, и, вкусный, место, идеально,...",очень красивый и вкусный место идеально подход...,mixed_ru_en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2307,Айтолкын Жанкелді,15,1,"Вчера были у вас, интерьер все круто. Но обслу...",371,2021-12-28,negative,"Вчера были у вас, интерьер все круто. Но обслу...",None,"Вчера были у вас, интерьер все круто. Но обслу...",ru,"[вчера, быть, у, вы, интерьер, всё, круто, но,...",вчера быть у вы интерьер всё круто но обслужив...,russian
2308,Артур Артур,14,1,"Пришли компанией в полупустой зал, ожидали око...",236,2021-12-26,negative,"Пришли компанией в полупустой зал, ожидали око...","[😂, 🧔🏻‍♂️, ✌🏻]","Пришли компанией в полупустой зал, ожидали око...","ru,en","[прислать, компания, в, полупустой, зал, ожида...",прислать компания в полупустой зал ожидать око...,mixed_ru_en
2309,Карина Сапашева,4,2,"Были здесь вчера с подругами, впечатления оста...",386,2021-12-26,negative,"Были здесь вчера с подругами, впечатления оста...",None,"Были здесь вчера с подругами, впечатления оста...",ru,"[быть, здесь, вчера, с, подруга, впечатление, ...",быть здесь вчера с подруга впечатление остатьс...,russian
2310,Sary Qyz,64,5,5 поставлю за вкусную кухню👍🏻обслуживание было...,169,2021-12-24,positive,5 поставлю за вкусную кухню👍🏻обслуживание было...,"[👍🏻, 😁, 😄]",5 поставлю за вкусную кухню👍🏻обслуживание было...,ru,"[5, поставить, за, вкусный, кухню👍🏻обслуживани...",5 поставить за вкусный кухню👍🏻обслуживание быт...,russian


In [275]:
from kaznlp.normalization.ininorm import Normalizer
from kaznlp.tokenization.tokhmm import TokenizerHMM
from kaznlp.morphology.analyzers import AnalyzerDD
from kaznlp.morphology import utils as kaz_utils
import os

# Инициализация
kaz_normalizer = Normalizer()
kaz_tokenizer = TokenizerHMM(
  model=os.path.join('/home/jovyan/work/nlp_2gis/kaznlp', 'kaznlp', 'tokenization', 'tokhmm.mdl')
)
kaz_analyzer = AnalyzerDD()
kaz_analyzer.load_model(
  os.path.join('/home/jovyan/work/nlp_2gis/kaznlp', 'kaznlp', 'morphology', 'mdl')
)

print("✓ KazNLP загружен и инициализирован!")

# Функция лемматизации казахских слов
def lemmatize_kazakh_word(word):
  """Лемматизация казахского слова - извлечение корня"""
  if not word or pd.isna(word):
      return word

  # Анализируем слово
  is_covered, analyses = kaz_analyzer.analyze(word.lower())

  if analyses and len(analyses) > 0:
      # Берём первый (наиболее вероятный) анализ
      analysis = analyses[0]
      # Извлекаем корень (root)
      root_info = kaz_utils.get_root(analysis, sdlm=' ', mdlm='_')
      # root_info - это словарь с ключами 'sf' (surface form), 'lp', 'rp'
      lemma = root_info['sf']
      return lemma
  else:
      # Слово не распознано анализатором - возвращаем как есть
      return word.lower()

# Тест
test_words = ['тағамдар', 'қонақ', 'күттік', 'дәмді']
print("Тестирование лемматизации казахских слов:\n")
for word in test_words:
  lemma = lemmatize_kazakh_word(word)
  print(f"{word:15} → {lemma}")

# Функция для обработки смешанных казахско-русских текстов
from pymorphy3 import MorphAnalyzer
morph_ru = MorphAnalyzer()

def lemmatize_mixed_kk_ru(text):
  """Лемматизация смешанного казахско-русского текста"""
  if pd.isna(text) or text == '':
      return None

  try:
      # Нормализация через kaznlp
      normalized = kaz_normalizer.normalize(
          str(text),
          translit=True,
          desegment=2,
          dedupe=2,
          emojiresolve=False
      )

      # Токенизация
      sentences = kaz_tokenizer.tokenize(normalized)
      all_lemmas = []

      for tokens in sentences:
          for token in tokens:
              # Пробуем лемматизировать как русское слово
              parsed_ru = morph_ru.parse(token)[0]

              if parsed_ru.score >= 0.5:  # Уверенно русское слово
                  all_lemmas.append(parsed_ru.normal_form)
              else:  # Скорее всего казахское слово
                  kaz_lemma = lemmatize_kazakh_word(token)
                  all_lemmas.append(kaz_lemma)

      return ' '.join(all_lemmas)

  except Exception as e:
      print(f"Ошибка: {e}")
      return text.lower()

# Тест на смешанном тексте
test_mixed = "Іші әдемі, интерьер эстетичный, тамақтары дәмді"
result = lemmatize_mixed_kk_ru(test_mixed)
print(f"\nТест смешанного текста:")
print(f"Вход:  {test_mixed}")
print(f"Выход: {result}")

✓ KazNLP загружен и инициализирован!
Тестирование лемматизации казахских слов:

тағамдар        → тағам
қонақ           → қонақ
күттік          → күт
дәмді           → дәмді
Ошибка: expected string or bytes-like object, got 'dict'

Тест смешанного текста:
Вход:  Іші әдемі, интерьер эстетичный, тамақтары дәмді
Выход: іші әдемі, интерьер эстетичный, тамақтары дәмді


In [279]:
# Простая функция для смешанных текстов - токены без лемматизации казахских слов
def process_mixed_text_simple(text):
  """Обработка смешанного текста: русские - лемматизация, казахские - просто токены"""
  if pd.isna(text) or text == '':
      return None

  try:
      # Нормализация
      normalized = kaz_normalizer.normalize(
          str(text),
          translit=False,
          desegment=2,
          dedupe=2,
          emojiresolve=False,
          stats=False
      )

      # Токенизация через kaznlp
      sentences = kaz_tokenizer.tokenize(normalized)
      all_tokens = []

      for tokens in sentences:
          for token in tokens:
              # Пропускаем пустые и пунктуацию
              if not token or len(token.strip()) == 0:
                  continue
              if token in '.,!?;:—-()[]{}«»""\'%':
                  continue

              # Пробуем как русское слово
              parsed_ru = morph_ru.parse(token)[0]

              if parsed_ru.score >= 0.5:  # Русское - лемматизируем
                  all_tokens.append(parsed_ru.normal_form)
              else:  # Казахское - оставляем как есть
                  all_tokens.append(token.lower())

      return ' '.join(all_tokens)

  except Exception as e:
      print(f"Ошибка: {e}")
      return text.lower()

# Тест
test_cases = [
  "Іші әдемі, интерьер эстетичный, тамақтары дәмді",
  "Туған күн иесіне -10% деп еді, істемедіі а так бары зоор",
  "Очень красивое и вкусное место, идеально подходит для свиданий"
]

print("=== Тест: русские слова - лемма, казахские - токены ===\n")
for i, text in enumerate(test_cases, 1):
  result = process_mixed_text_simple(text)
  print(f"{i}. Вход:  {text[:65]}")
  print(f"   Выход: {result[:65]}")
  print()

=== Тест: русские слова - лемма, казахские - токены ===

1. Вход:  Іші әдемі, интерьер эстетичный, тамақтары дәмді
   Выход: іші әдемі интерьер эстетичный тамақтары дәмді

2. Вход:  Туған күн иесіне -10% деп еді, істемедіі а так бары зоор
   Выход: туғать күн иесіне 1 0 деп еді істемедіі а так бар зоор

3. Вход:  Очень красивое и вкусное место, идеально подходит для свиданий
   Выход: очень красивый и вкусный место идеально подходить для свидание



In [285]:
# Проверяем, где пусто или казахские тексты
needs_processing = df[
  (df['text_lemmatized'].isna()) |
  (df['text_type'].isin(['mixed_kk_ru', 'mixed_ru_en', 'translit']))
]

print(f"Текстов для простой токенизации: {len(needs_processing)}")
print(f"\nТипы текстов:")
print(needs_processing['text_type'].value_counts())

# Простая функция токенизации для казахских текстов
def simple_tokenize(text):
  """Простая токенизация без лемматизации"""
  if pd.isna(text) or text == '':
      return None

  try:
      # Нормализация через kaznlp
      normalized = kaz_normalizer.normalize(
          str(text),
          translit=False,
          desegment=2,
          dedupe=2,
          emojiresolve=False,
          stats=False
      )

      # Токенизация
      sentences = kaz_tokenizer.tokenize(normalized)
      all_tokens = []

      for tokens in sentences:
          for token in tokens:
              # Только буквы, пропускаем пунктуацию
              if token and any(c.isalpha() for c in token):
                  all_tokens.append(token.lower())

      return ' '.join(all_tokens)

  except:
      # Если ошибка - простая токенизация через split
      return ' '.join([w.lower() for w in str(text).split() if w.strip()])

# Тест
test_text = "Туған күн иесіне -10% деп еді"
result = simple_tokenize(test_text)
print(f"Тест: {test_text}")
print(f"Результат: {result}")

# Применяем к казахским/необработанным текстам
print("Обработка казахских/необработанных текстов...")

mask = (df['text_lemmatized'].isna()) | (df['text_type'].isin(['mixed_kk_ru', 'mixed_ru_en', 'translit']))

# Применяем простую токенизацию
df.loc[mask, 'tokens'] = df.loc[mask, 'text_normalized'].apply(simple_tokenize)

print(f"✓ Обработано: {mask.sum()} текстов")
print(f"✓ Всего текстов с text_lemmatized: {df['tokens'].notna().sum()} из {len(df)}")

# Проверяем результат
print("\n=== Примеры обработанных текстов ===")
for idx, row in df[mask].head(3).iterrows():
  print(f"\nТип: {row['text_type']}")
  print(f"Оригинал: {row['text_normalized'][:60]}")
  print(f"Токены:   {row['tokens'][:60]}")

Текстов для простой токенизации: 299

Типы текстов:
text_type
mixed_kk_ru    231
mixed_ru_en     67
translit         1
Name: count, dtype: int64
Тест: Туған күн иесіне -10% деп еді
Результат: туған күн иесіне деп еді
Обработка казахских/необработанных текстов...
✓ Обработано: 299 текстов
✓ Всего текстов с text_lemmatized: 2312 из 2312

=== Примеры обработанных текстов ===

Тип: mixed_ru_en
Оригинал: Очень красивое и вкусное место, идеально подходит для свидан
Токены:   очень красивое и вкусное место идеально подходит для свидани

Тип: mixed_kk_ru
Оригинал: Туған күн иесіне -10% деп еді, істемедіі 🙄🙄🙄 а так бары зоор
Токены:   туған күн иесіне деп еді істемедіі а так бары зоор

Тип: mixed_kk_ru
Оригинал: Іші әдемі, интерьер эстетичный, тамақтары дәмді. но өте ұзақ
Токены:   іші әдемі интерьер эстетичный тамақтары дәмді но өте ұзақ уа


In [282]:
df.to_csv('reviews_cleaned_final.csv', index=False)

In [301]:
df[df['languages'] == 'ru,kz']

,author,author_reviews_count,rating,text,text_length,date,sentiment,text_1,emojis,text_normalized,languages,tokens,text_lemmatized,text_type
5,Sailauovaaas_ _,1,4,"Туған күн иесіне -10% деп еді, істемедіі 🙄🙄🙄 а...",62,2025-09-30,positive,"Туған күн иесіне -10% деп еді, істемедіі 🙄🙄🙄 а...","['🙄', '🙄', '🙄', '😋']","Туған күн иесіне -10% деп еді, істемедіі 🙄🙄🙄 а...","ru,kz",[],NaN,mixed_kk_ru
6,Каракат Кайраткызы,2,5,"Іші әдемі, интерьер эстетичный, тамақтары дәмд...",111,2025-09-29,positive,"Іші әдемі, интерьер эстетичный, тамақтары дәмд...",NaN,"Іші әдемі, интерьер эстетичный, тамақтары дәмд...","ru,kz",[],NaN,mixed_kk_ru
10,Saule Kokanaeva,2,5,Қайырлы күн! кеше сіздерде туған күн тойладық....,206,2025-09-27,positive,Қайырлы күн! кеше сіздерде туған күн тойладық....,"['❤️', '😍', '🌺']",Қайырлы күн! кеше сіздерде туған күн тойладық....,"ru,kz",[],NaN,mixed_kk_ru
16,Рауан Манат,2,5,Өте керемет кафе екен тағыда келуге турарлык ө...,188,2025-09-23,positive,Өте керемет кафе екен тағыда келуге турарлык ө...,NaN,Өте керемет кафе екен тағыда келуге турарлык ө...,"ru,kz",[],NaN,mixed_kk_ru
61,Alibi Adilkhanuly,4,5,"Бәрі күшті, ұнады!\nПланшетпен менюден тапсыры...",66,2025-08-28,positive,"Бәрі күшті, ұнады! Планшетпен менюден тапсырыс...",['👍🏻'],"Бәрі күшті, ұнады! Планшетпен менюден тапсырыс...","ru,kz",[],NaN,mixed_kk_ru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182,Islam Bodaubek,1,5,"Рахмет сіздерге тағамдарыңыз дәмді екен, атмос...",110,2022-08-12,positive,"Рахмет сіздерге тағамдарыңыз дәмді екен, атмос...",NaN,"Рахмет сіздерге тағамдарыңыз дәмді екен, атмос...","ru,kz",[],NaN,mixed_kk_ru
2238,Айтолқын Жанысбек,18,4,Тамақтардың дәмдері жақсы бірақ порциясы кішке...,135,2022-05-09,positive,Тамақтардың дәмдері жақсы бірақ порциясы кішке...,"['😊', '❤️']",Тамақтардың дәмдері жақсы бірақ порциясы кішке...,"ru,kz",[],NaN,mixed_kk_ru
2282,Asylzat Asylzat,16,5,Тамақтары дәмді😍👍🏻 Обслуживание вообще топ👍🏻 Х...,116,2022-02-11,positive,Тамақтары дәмді😍👍🏻 Обслуживание вообще топ👍🏻 Х...,"['😍', '👍🏻', '👍🏻', '👍🏻', '😍']",Тамақтары дәмді😍👍🏻 Обслуживание вообще топ👍🏻 Х...,"ru,kz",[],NaN,mixed_kk_ru
2290,Aidana Nurseyt,2,5,Бүгін подругаларыммен осы заведениеде болдық. ...,595,2022-02-06,positive,Бүгін подругаларыммен осы заведениеде болдық. ...,"['👍🏻', '🙂', '🙂', '🤲🏻', '🙂', '👍🏻', '🤗']",Бүгін подругаларыммен осы заведениеде болдық. ...,"ru,kz",[],NaN,mixed_kk_ru


In [299]:
df = pd.read_csv('reviews_cleaned_final.csv')

In [294]:

# Проверяем, где text_lemmatized пустой
mask = df['text_lemmatized'].isna()
print(f"Текстов без text_lemmatized: {mask.sum()}")

# Смотрим на примеры
if mask.sum() > 0:
  print("\nПримеры необработанных:")
  for idx, row in df[mask].head(3).iterrows():
      print(f"\nТип: {row['text_type']}")
      print(f"Tokens: {row['tokens'][:50] if isinstance(row['tokens'], list) else row['tokens']}")
      print(f"Text_lemmatized: {row['text_lemmatized']}")

# Для пустых text_lemmatized просто берём tokens и склеиваем
mask = df['text_lemmatized'].isna()

if mask.sum() > 0:
  # Склеиваем токены в строку через пробел
  df.loc[mask, 'text_lemmatized'] = df.loc[mask, 'tokens'].apply(
      lambda x: ' '.join(x) if isinstance(x, list) else str(x).lower()
  )

  print(f"✓ Обработано: {mask.sum()} текстов")
  print(f"✓ Всего с text_lemmatized: {df['text_lemmatized'].notna().sum()} из {len(df)}")

  # Проверяем результат
  print("\n=== Примеры ===")
  for idx, row in df[mask].head(3).iterrows():
      print(f"\nТип: {row['text_type']}")
      print(f"Text_lemmatized: {row['text_lemmatized'][:70]}")
else:
  print("✓ Все тексты уже обработаны!")

Текстов без text_lemmatized: 233

Примеры необработанных:

Тип: mixed_kk_ru
Tokens: []
Text_lemmatized: nan

Тип: mixed_kk_ru
Tokens: []
Text_lemmatized: nan

Тип: mixed_kk_ru
Tokens: []
Text_lemmatized: nan
✓ Обработано: 233 текстов
✓ Всего с text_lemmatized: 2312 из 2312

=== Примеры ===

Тип: mixed_kk_ru
Text_lemmatized: []

Тип: mixed_kk_ru
Text_lemmatized: []

Тип: mixed_kk_ru
Text_lemmatized: []


In [300]:
df

,author,author_reviews_count,rating,text,text_length,date,sentiment,text_1,emojis,text_normalized,languages,tokens,text_lemmatized,text_type
0,Арайлым Куандык,2,5,От всей души благодарим официантку Гулжайну за...,150,2025-10-05,positive,От всей души благодарим официантку Гулжайну за...,['❤️\u200d🔥'],От всей души благодарим официантку Гулжайну за...,ru,"['от', 'весь', 'душа', 'благодарить', 'официан...",от весь душа благодарить официантка гулжайн за...,russian
1,Merey ♥️,16,1,Обслуживание оставляет желать лучшего. Персона...,100,2025-10-05,negative,Обслуживание оставляет желать лучшего. Персона...,NaN,Обслуживание оставляет желать лучшего. Персона...,ru,"['обслуживание', 'оставлять', 'желать', 'хорош...",обслуживание оставлять желать хороший персонал...,russian
2,Adina Nursapina,1,5,"очень милый персонал,осталась очень довольной😍😍😍",48,2025-10-03,positive,"очень милый персонал,осталась очень довольной😍😍😍","['😍', '😍', '😍']","очень милый персонал,осталась очень довольной😍😍😍",ru,"['очень', 'милый', 'персонал', 'остаться', 'оч...",очень милый персонал остаться очень довольной😍😍😍,russian
3,Zhigerkizi demi,41,5,Реально вкусно и суперские милкшейки,36,2025-10-03,positive,Реально вкусно и суперские милкшейки,NaN,Реально вкусно и суперские милкшейки,ru,"['реально', 'вкусно', 'и', 'суперский', 'милкш...",реально вкусно и суперский милкшейк,russian
4,Михаил Класс,21,5,"Очень красивое и вкусное место, идеально подхо...",225,2025-10-03,positive,"Очень красивое и вкусное место, идеально подхо...",NaN,"Очень красивое и вкусное место, идеально подхо...","ru,en","['очень', 'красивый', 'и', 'вкусный', 'место',...",очень красивый и вкусный место идеально подход...,mixed_ru_en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2307,Айтолкын Жанкелді,15,1,"Вчера были у вас, интерьер все круто. Но обслу...",371,2021-12-28,negative,"Вчера были у вас, интерьер все круто. Но обслу...",NaN,"Вчера были у вас, интерьер все круто. Но обслу...",ru,"['вчера', 'быть', 'у', 'вы', 'интерьер', 'всё'...",вчера быть у вы интерьер всё круто но обслужив...,russian
2308,Артур Артур,14,1,"Пришли компанией в полупустой зал, ожидали око...",236,2021-12-26,negative,"Пришли компанией в полупустой зал, ожидали око...","['😂', '🧔🏻\u200d♂️', '✌🏻']","Пришли компанией в полупустой зал, ожидали око...","ru,en","['прислать', 'компания', 'в', 'полупустой', 'з...",прислать компания в полупустой зал ожидать око...,mixed_ru_en
2309,Карина Сапашева,4,2,"Были здесь вчера с подругами, впечатления оста...",386,2021-12-26,negative,"Были здесь вчера с подругами, впечатления оста...",NaN,"Были здесь вчера с подругами, впечатления оста...",ru,"['быть', 'здесь', 'вчера', 'с', 'подруга', 'вп...",быть здесь вчера с подруга впечатление остатьс...,russian
2310,Sary Qyz,64,5,5 поставлю за вкусную кухню👍🏻обслуживание было...,169,2021-12-24,positive,5 поставлю за вкусную кухню👍🏻обслуживание было...,"['👍🏻', '😁', '😄']",5 поставлю за вкусную кухню👍🏻обслуживание было...,ru,"['5', 'поставить', 'за', 'вкусный', 'кухню👍🏻об...",5 поставить за вкусный кухню👍🏻обслуживание быт...,russian
